# MVSep MDX23 Colab v2.1.1
* Slow separations fixed, unified installation cells (28.12.23)
* Chunks currently set to 100000 works with 3:00 track. Decrease it if you run out of memory
* With current settings it takes 35 minutes to separate 3:00 track. For faster separation set both overlaps to 0.8 as it's still good performance/quality parameter (0.96 max tested, with 300K before memory issues).
<br><br>
* Don't write file names in input below - only folder paths allowed (otherwise it will show just /content/MVSEP-MDX23-Colab_v2.1).
* Batch processing - all audio files inside provided input folder will be separated.<br>
* Not all EOFError errors are critical. Check your output folder after separation first.<br>
* [v2.2 final](https://colab.research.google.com/github/jarredou/MVSEP-MDX23-Colab_v2/blob/v2.2/MVSep-MDX23-Colab.ipynb) (with faster optional 2 stem output) - better SDR, more vocal residues

Colab version of MDX23 algorithm from [MVSEP.COM](https://www.mvsep.com) with some tweaks:
* Updated with new UVR-MDX': voc_ft & Instr-HQ3 models
* Fixed high frequency bleed in vocals
* Fixed volume compensation for MDX models
* It receives input files from temp Colab disk (open file manager in the left to upload a file), not from GDrive, but you can change it to GDrive by setting /content/drive/MyDrive/input (be aware that the path in some cases is case-sensitive and you must create that folder on your own)
* For a bit better quality, chunks_size can be set to 500K for ~2:41-3:58 tracks - 500K fails with 5 minute tracks
* Replace inference.py by [that](https://raw.githubusercontent.com/Infisrael/MVSEP-MDX23-Colab_v2.1/main/inference.py) one if you still have memory issues with 5 minute tracks and chunk_size set to 300K (open file manager and go to Colab folder) - be aware that it disables Demucs denoiser and increases vocal residues.
* Beta 2.2 1.5.1 [inference](https://cdn.discordapp.com/attachments/887455924845944873/1129470812580225164/inference_1.5.1_vocft_mod.py) - might be a bit cleaner (denoiser still on).
* 2.2 Pre Beta 3 /wo v3 MDX yet [Colab](https://colab.research.google.com/github/jarredou/MVSEP-MDX23-Colab_v2/blob/597b5b7f653e4593a0a94938a3923077d66f8767/MVSep-MDX23-Colab.ipynb) and [inference](https://drive.google.com/file/d/1bpZKZynmdsYcriF-M8t8yLtVLm7zRz5U/view?usp=sharing) - might be even cleaner (denoiser still on) - delete two bigshifts and vocals only references in the cell to make it work.
* Beta 2.2 1.5.2 (shifts 0, less muddy) - [inference](https://drive.google.com/file/d/1GKTShwgVKwyLNLssh2AmLiN5vEwrUPus/view?usp=sharing).
* For both overlaps set to 0.95 and 500K chunks it takes 27 minutes to process 3:58 track with shifts 0 set in inference from above.
* It will show a lot of warinings and exceptions (e.g. PySoundFile failed, EOFError) frequently, but separation can be successful with these errors (and unsuccessful too, if you set wrong parameters)
* Instrum is inverted vocals stem
* Instrum2 is the sum of drums+bass+other stems (muddier, smaller SDR)
* It doesn't use Demucs denoiser disabled yet, so it's less noisy here than current 2.1 - that fix in 2.1 was introduced after the 2.1 release in 2.0 repo (fixes problems when no matter what chunks you use, you still get memory errors in e.g. Demucs 3/4 step)


Credits:
* [https://github.com/ZFTurbo/MVSEP-MDX23-music-separation-model](https://github.com/ZFTurbo/MVSEP-MDX23-music-separation-model)
* Models by [Demucs](https://github.com/facebookresearch/demucs), [UVR GUI Team](https://github.com/Anjok07/ultimatevocalremovergui) - Anjok / Aufr33 & [Kimberley Jensen](https://github.com/KimberleyJensen)
* Adaptation & tweaks by [jarredou](https://github.com/jarredou/MVSEP-MDX23-Colab_v2/)

In [ ]:
#@markdown #Installation
#@markdown *Run this cell to install MVSep-MDX23*
print('Installing... This will take 1 minute...')
%cd /content
from google.colab import drive
drive.mount('/content/drive')
!git clone https://github.com/deton24/MVSEP-MDX23-Colab_v2.1 &> /dev/null
%cd /content/MVSEP-MDX23-Colab_v2.1
!pip install -r requirements.txt &> /dev/null
!pip install demucs
!pip install onnxruntime-gpu
# onnxruntime-gpu nightly fix for cuda12.2
!python -m pip install ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/
print('Installation done !')

In [ ]:
#@markdown #Separation
from pathlib import Path
import glob

%cd /content/MVSEP-MDX23-Colab_v2.1
def console(t):
    get_ipython().system(t)

#file_path = '/content/drive/MyDrive/mvsep_dataset/' #@param {type:"string"}
folder_path = '/content/drive/MyDrive/input' #@param {type:"string"}
output_folder = '/content/drive/MyDrive/output' #@param {type:"string"}
overlap_large = 0.9 #@param {type:"slider", min:0.1, max:1, step:0.05}
overlap_small = 0.9 #@param {type:"slider", min:0.1, max:1, step:0.05}

#@markdown *Use lower chunk_size if you have memory errors*
chunk_size = 100000 #@param {type:"slider", min:50000, max:1000000, step:100000}

#console(f'python inference.py --large_gpu --chunk_size {chunk_size} --input_audio "{file_path}" --output_folder "./results/{filename}"')
for file_path in glob.glob(folder_path+"/*"):
  filename =  Path(file_path).stem
  Path(output_folder,filename).mkdir(parents=True, exist_ok=True)
  console(f'python inference.py --large_gpu --chunk_size {chunk_size} --input_audio "{file_path}" --overlap_large {overlap_large} --overlap_small {overlap_small} --output_folder "{output_folder}"/"{filename}"')


In [ ]:
#@title (optional) Mixdown to 64 bit (might be slightly better instrum2 equivalent (32-bit float) - can be placebo in most cases; (change it to pcm_s16le for 16 bit). Write file name without input file extension in paths below

!ffmpeg -i "/content/drive/MyDrive/output/your track/your track_bass.wav" -i "/content/drive/MyDrive/output/your track/your track_other.wav" -i "/content/drive/MyDrive/output/your track/your track_drums.wav" -filter_complex "[0]volume=3[a];[1]volume=3[b];[2]volume=3[c];[a][b][c]amix=inputs=3:duration=longest" -c:a pcm_f64le '/content/drive/MyDrive/output/output.wav'